In [1]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

# util
import pandas as pd
import csv, time
from datetime import datetime

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [2]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

In [3]:
def generate_url(request_id="", action=""):
    url = 'https://services.sentinel-hub.com/batch/v1/process/'
    if request_id:
        url += f'{request_id}/'
        if action:
            url += f'{action}'
    return url

In [4]:
old_rids = set()
active_rids = set()

while True:
    # Query REST endpoint for all requests
    viewtoken = 0
    requests = []
    while viewtoken is not None:
        response = oauth.request('GET', generate_url()+f'?viewtoken={viewtoken}').json()
        requests.extend(response['member'])
        viewtoken=response['view']['nextToken']
    
    active_rids = set([r['id'] for r in requests if r['status'] == 'PROCESSING' and r['userAction'] == 'START'])
    print('active:', active_rids, datetime.today().isoformat())
    
    # Log end time of finished requests
    finished_rids = old_rids - active_rids
    for frid in finished_rids:
        status = oauth.request('GET', generate_url(frid)).json()['status']
        if status == 'DONE':
            with open('endtimestamps.csv', 'a') as file:
                writer = csv.writer(file)
                writer.writerow([frid,datetime.today().isoformat()]) # log end time
                
    # Outdate active request ids
    old_rids = active_rids
    print('old:', old_rids)
    
    time.sleep(1)

active: {'4293b3d0-afba-4df3-b7c9-6b0ed9725ea4'} 2020-04-24T14:29:04.050839
old: {'4293b3d0-afba-4df3-b7c9-6b0ed9725ea4'}
active: {'4293b3d0-afba-4df3-b7c9-6b0ed9725ea4'} 2020-04-24T14:29:05.354117
old: {'4293b3d0-afba-4df3-b7c9-6b0ed9725ea4'}
active: {'4293b3d0-afba-4df3-b7c9-6b0ed9725ea4'} 2020-04-24T14:29:06.449322
old: {'4293b3d0-afba-4df3-b7c9-6b0ed9725ea4'}


KeyboardInterrupt: 

In [ ]:
# pd.Series.value_counts([(r['status'], r['userAction']) for r in requests])